In [1]:
from datetime import date

import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [3]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

with etl_conn.connect() as conn:
    

# Extract

In [4]:
dim_ips = pd.read_sql_table('ips', co_sa)
dim_ips.info()


# Transformations

In [5]:
dim_ips.replace({'':'0'},inplace=True)

In [9]:
dim_ips["saved"] = date.today()
dim_ips.describe(include='all')

# load

In [11]:
dim_ips.to_sql('dim_ips', etl_conn, if_exists='replace',index_label='key_dim_ips')